In [1]:
import os

In [2]:
import sqlalchemy as db

In [3]:
from sqlalchemy import create_engine

In [4]:
from uuid import uuid4

In [5]:
from faker import Faker

In [6]:
fake = Faker()

In [7]:
from random import randint

## 15.1 
### Write a method to find the number of employees in each department 

There are two tables:

```sql
create table departments (
    id string primary key,
    name string
)
```

```sql
create table employees (
    id string primary key,
    name string,
    department_id string references departments(id)
)
```

```sql
create table groups (
    id string primary key,
    name string                      
)
```

```sql
create table group_members (
    group_id string references groups(id) not null,
    employee_id string references employees(id) not null,
    unique (group_id, employee_id)
)
```

Query

```sql
select d.name, count(*) as 'num_employees' from departments d join employees e on d.id=e.department_id group by d.name;
```

In [8]:
os.remove('foo.db')
engine = create_engine('sqlite:///foo.db')
connection = engine.connect()

## Create Tables

In [9]:
# Department
engine.execute(f"create table departments (id string primary key, name string)")

In [10]:
# Employees
engine.execute(f"create table employees (id string primary key, name string, department_id string references departments(id))")

In [11]:
# Groups
engine.execute(f"create table groups (id string primary key, name string)")

In [12]:
# Group Members
engine.execute(f"create table group_members (group_id string references groups(id) not null, employee_id string references employees(id) not null, unique (group_id, employee_id))")

In [13]:
departments = [
    (str(uuid4()), 'Human Resource', True),
    (str(uuid4()), 'Support', True),
    (str(uuid4()), 'Sales', True),
    (str(uuid4()), 'Technology', False)
]

In [14]:
for (id, name, add_people) in departments:
    engine.execute(f"insert into departments (id, name) values ('{id}', '{name}')")
    if add_people:
        for index in range(0, randint(1, 10)):
            engine.execute(f"insert into employees (id, name, department_id) values ('{str(uuid4())}', '{fake.name()}', '{id}')")

In [15]:
# Employees
employees_ids = [r[0] for r in engine.execute(f"select e.id, e.name, d.name from employees e join departments d on e.department_id=d.id")]

In [16]:
for job in [fake.job() for index in range(0, randint(1, 10))]:
    group_id = str(uuid4())
    engine.execute(f"insert into groups (id, name) values ('{group_id}', '{job}')")
    ids = [(group_id, employees_ids[randint(0, len(employees_ids)-1)]) for i in range(0, 10)]
    string = ','.join(f"('{i[0]}', '{i[1]}')" for i in ids)
    engine.execute(f"insert into group_members (group_id, employee_id) values {string} on conflict (group_id, employee_id) do nothing")

In [17]:
# Show me all the department
[r for r in engine.execute(f"select * from departments")]

[('8dc4a390-c7c5-418a-a379-4d07719ad76b', 'Human Resource'),
 ('c73f0c7b-d786-4f5a-8799-07169d638f52', 'Support'),
 ('756e228b-3c50-4d3d-9726-34608fb7dd9e', 'Sales'),
 ('1b559f55-8107-4620-8f86-1c9433be8a8b', 'Technology')]

In [18]:
employees_ids

['71a3796b-8575-4d5e-9d7b-6c79be5be721',
 'c41cfcc7-e2d1-474a-a96a-f6f23c062b4e',
 '2d4a5549-7ce7-4948-ae0e-6bff54b44b26',
 'bae4f2d0-0276-4fe5-b3cf-f96e3fca0576',
 '4ba13ef2-42de-4714-8aa9-8e27bba47486',
 'ddde6e96-b0af-476b-bfb0-5ece6574b1b4',
 '89517ec3-61a0-4ea9-91f1-d428b6fd4595',
 'eb2e5970-4322-47f5-8318-39193518c205',
 '55ff5f0a-f8f1-4bf5-8a4b-8f18687141ae',
 'eb2dc2bd-dbe9-4471-ace4-f3aed852d601',
 'cea488d9-fdc2-4962-93de-4cf73b9f82ed',
 '47e2dafa-c98c-46c7-9ad8-c61745ec61e5',
 'd9c5d302-4290-4873-8282-7a5aa53fed21']

In [19]:
[r for r in engine.execute(f"select * from groups")]

[('7649a6e9-e437-4f1f-a250-8a49e42cd6f8', 'Engineer, building services'),
 ('467423c6-49f1-41e8-aecf-262f1f451564', 'Music therapist')]

In [20]:
[r for r in engine.execute(f"select * from group_members limit 10")]

[('7649a6e9-e437-4f1f-a250-8a49e42cd6f8', '89517ec3-61a0-4ea9-91f1-d428b6fd4595'),
 ('7649a6e9-e437-4f1f-a250-8a49e42cd6f8', '4ba13ef2-42de-4714-8aa9-8e27bba47486'),
 ('7649a6e9-e437-4f1f-a250-8a49e42cd6f8', '55ff5f0a-f8f1-4bf5-8a4b-8f18687141ae'),
 ('7649a6e9-e437-4f1f-a250-8a49e42cd6f8', 'd9c5d302-4290-4873-8282-7a5aa53fed21'),
 ('7649a6e9-e437-4f1f-a250-8a49e42cd6f8', 'ddde6e96-b0af-476b-bfb0-5ece6574b1b4'),
 ('7649a6e9-e437-4f1f-a250-8a49e42cd6f8', '2d4a5549-7ce7-4948-ae0e-6bff54b44b26'),
 ('467423c6-49f1-41e8-aecf-262f1f451564', '71a3796b-8575-4d5e-9d7b-6c79be5be721'),
 ('467423c6-49f1-41e8-aecf-262f1f451564', '89517ec3-61a0-4ea9-91f1-d428b6fd4595'),
 ('467423c6-49f1-41e8-aecf-262f1f451564', 'cea488d9-fdc2-4962-93de-4cf73b9f82ed'),
 ('467423c6-49f1-41e8-aecf-262f1f451564', 'bae4f2d0-0276-4fe5-b3cf-f96e3fca0576')]

In [21]:
# Need left join to add the null join case
[r for r in engine.execute(f"select d.id, d.name, count(e.id) as 'num_employees' from departments d left join employees e on d.id=e.department_id group by d.id, d.name")]

[('1b559f55-8107-4620-8f86-1c9433be8a8b', 'Technology', 0),
 ('756e228b-3c50-4d3d-9726-34608fb7dd9e', 'Sales', 10),
 ('8dc4a390-c7c5-418a-a379-4d07719ad76b', 'Human Resource', 1),
 ('c73f0c7b-d786-4f5a-8799-07169d638f52', 'Support', 2)]

In [22]:
[r for r in engine.execute(f"select g.name, e.name from groups g join group_members gm on g.id=gm.group_id join employees e on e.id=gm.employee_id")]

[('Engineer, building services', 'Joseph Williams'),
 ('Engineer, building services', 'Michael Clark'),
 ('Engineer, building services', 'Kathleen Molina'),
 ('Engineer, building services', 'Donald Alvarado'),
 ('Engineer, building services', 'Patrick Paul'),
 ('Engineer, building services', 'George Becker'),
 ('Music therapist', 'Jeremy Wright'),
 ('Music therapist', 'Joseph Williams'),
 ('Music therapist', 'Mary Miller'),
 ('Music therapist', 'Henry Chambers'),
 ('Music therapist', 'Patrick Barr'),
 ('Music therapist', 'Kathleen Molina'),
 ('Music therapist', 'Patrick Paul')]

In [23]:
#connection.close()

## 15.2 
### What are the different types of joins? Please explain how they differ and why certain types are better in certain situations

### Join (Inner)

![Inner Join](https://www.techonthenet.com/sql/images/inner_join.gif)

Join only the intersection where the join condition meets.

*When to use?*
Most common join, where you only want exists.

### Left (Outer) Join

![Left Join](https://www.techonthenet.com/sql/images/left_outer_join.gif)

Join everything on the left, even null.

*When to use?*


### Right (Right) Join

![Right Join](https://www.techonthenet.com/sql/images/right_outer_join.gif)

Join everything on the right, even null.

*When to use?*

### Full (Outer) Join

![Full Join](https://www.techonthenet.com/sql/images/full_outer_join.gif)

Give me everything.

*When to use?*

## 15.3 
### What is denormalization? Explain the pros and cons 

Denomorlization is when you want to make sure data does not repeat itself.

*Pros*
* ACID compliance requirements
* Don't Repeat Yourself (DRY) - data integrity
* Data needs to be change only once (consistent) - it resides in one place
* size efficiency
* inserts and updates run quickly


*Cons*
Joins can be expensive.
Read can be expensive
Queries can by more complicated
With join, indexes does not work as efficientyly

## 15.4 
### Draw an entity-relationship diagram for a database with companies, people, and pro- fessionals (people who work for companies)

**see above**

## 15.5 
Imagine a simple database storing information for students’ grades. Design what this database might look like, and provide a SQL query to return a list of the honor roll students (top 10%), sorted by their grade point average 

Two tables

```sql
create table students (
    id string primary key,
    name string
    
)
```

```sql
create table grades (
    id string primary key,
    student_id string references students(id),
    grade integer not null
)
```


In [24]:
engine.execute(f"create table students (id string primary key, name string)")
engine.execute(f"create table grades (id string primary key, student_id string references students(id), grade integer not null)")

In [25]:
for (id, name) in [(uuid4(), fake.name()) for index in range(0, 125)]:
    engine.execute(f"insert into students (id, name) values ('{id}', '{name}')")
    for (grade_id, grade) in [(str(uuid4()), randint(0, 101)) for index in range(0, randint(1, 10))]:
        engine.execute(f"insert into grades (id, student_id, grade) values ('{grade_id}', '{id}', '{grade}')")

In [26]:
[r for r in engine.execute('select student_id, avg(grade) as gpa from grades')]

[('9ca33011-b6fc-44fc-b85c-2af3a65aab67', 51.44082840236686)]

In [27]:
[r for r in engine.execute('select count(id) from students limit 1')]

[(125,)]

In [28]:
[r for r in engine.execute('select s.name, g.id, avg(grade) as gpa from grades g left join students s on s.id=g.student_id group by s.name order by gpa desc limit (select count(id)/10 from students)')]

[('Susan Melton', '9636c7e8-f257-4ee6-82e8-fbfa718e221b', 94.0),
 ('Stephanie Erickson', 'df24877d-cab3-476a-a720-ab893b7a74da', 91.0),
 ('Brian Adams', 'd6a6de97-9480-427e-95e1-cbb9c39bfed5', 89.66666666666667),
 ('Thomas Joyce', '26de6f5e-7357-4710-9ab3-97947abb9a91', 79.25),
 ('Kathleen Obrien', 'e3de5178-c273-4acf-8f91-87b38716e08a', 76.11111111111111),
 ('Patricia Jones', 'ba3bd697-cee2-453f-b29b-5ea02f59dbec', 76.0),
 ('Ashley Fisher', '7a1fd2fb-b68d-42c1-b469-c74908e2bd0b', 74.75),
 ('Michael Martinez', '3833910c-9437-4ae2-ad7b-e55663278037', 74.66666666666667),
 ('Melissa Martinez', '04bf133f-e90f-4a05-934c-4f4be7c62cd9', 73.0),
 ('Steven Estes', '0e3a9e0e-9dba-4837-99ff-07f4c8d71f98', 73.0),
 ('Julie Rodriguez', '72472ed9-9792-481c-a0a3-1634365df81b', 68.5),
 ('Sandra Mccarthy', '4488ece1-3b57-4c46-81c2-23efce0dfa6b', 67.22222222222223)]